In [1]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

In [2]:
data_path=None
all_variables=None
test_path=None
# target='search_term'
target=None
nthreads=1 
min_mem_size=6 
run_time=1000
classification=True
scale=True
max_models=20    
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
analysis=0

In [3]:
#Funtion for meta data
def set_meta_data(analysis,run_id,server,data,test,model_path,target,run_time,classification,scale,model,balance,
                  balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['test_path']=test
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data




In [4]:
#Function to convert dictionary into json file
def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
    
#Function for random alphabet   
def alphabet(n):
    alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
    str=''
    r=len(alpha)-1   
    while len(str)<n:
        i=random.randint(0,r)
        str+=alpha[i]   
        return str

def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


    

def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val
    return d    

In [5]:
def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id  
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    

In [6]:
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm


def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    

In [7]:
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()     
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d

In [8]:

    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d

In [9]:
all_variables=None

In [10]:
data_path='C:/Users/pkash/Downloads/adult.csv'

In [11]:
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

# run_id to std out
print (run_id)

L


In [12]:
##For logs
logfile=run_id+'_autoh2o_log.zip'
logs_path=os.path.join(run_dir,'logs')
print(logs_path,' ',logfile)

C:\Users\pkash\L\logs   L_autoh2o_log.zip


In [13]:
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.201-b09, mixed mode)
  Starting server from C:\Users\pkash\Anaconda\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\pkash\AppData\Local\Temp\tmp6zbwsfwe
  JVM stdout: C:\Users\pkash\AppData\Local\Temp\tmp6zbwsfwe\h2o_pkash_started_from_python.out
  JVM stderr: C:\Users\pkash\AppData\Local\Temp\tmp6zbwsfwe\h2o_pkash_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,13 days
H2O cluster name:,H2O_from_python_pkash_lqu1pj
H2O cluster total nodes:,1
H2O cluster free memory:,1.743 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [14]:
# meta data
meta_data = set_meta_data(analysis, run_id,server_path,data_path,test_path,model_path,target,run_time,classification,scale,max_models,balance_y,balance_threshold,name,run_dir,nthreads,min_mem_size)
print(meta_data)

{'start_time': 1555262994.0324767, 'target': None, 'server_path': 'C:\\Users\\pkash', 'data_path': 'C:/Users/pkash/Downloads/adult.csv', 'test_path': None, 'max_models': 9, 'run_time': 1000, 'run_id': 'L', 'scale': False, 'classification': False, 'model_path': None, 'balance': False, 'balance_threshold': 0.2, 'project': None, 'end_time': 1555262994.0324767, 'execution_time': 0.0, 'run_path': 'C:\\Users\\pkash\\L', 'nthreads': 1, 'min_mem_size': 6, 'analysis': 0}


In [15]:
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [16]:
df.head()

age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [17]:

df.describe()

Rows:48842
Cols:15




,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
type,int,enum,int,enum,int,enum,enum,enum,enum,enum,int,int,int,enum,enum
mins,17.0,,12285.0,,1.0,,,,,,0.0,0.0,1.0,,
mean,38.643585438761704,,189664.13459727284,,10.078088530363212,,,,,,1079.0676262233324,87.50231358257237,40.42238237582409,,
maxs,90.0,,1490400.0,,16.0,,,,,,99999.0,4356.0,99.0,,
sigma,13.710509934443555,,105604.02542315728,,2.570972755592256,,,,,,7452.019057655393,403.00455212435907,12.391444024252307,,
zeros,0,,0,,0,,,,,,44807,46560,0,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K


In [18]:
# dependent variable
# assign target and inputs for classification or regression
if target==None:
    target=df.columns[14]   
y = target
y

'income'

In [19]:
x = list(df.columns)
x.remove(y)
x
# Add independent variables

meta_data['x']=x 
# impute missing values

_=impute_missing_values(df,x, scale)


In [20]:
# # impute missing values

# _ = df[ints].impute(method='median')

# if scale:
    
#     df[ints] = df[ints].scale()

In [21]:
if analysis == 3:
  classification=False
elif analysis == 2:
  classification=True
elif analysis == 1:
  classification=True

In [22]:
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok

In [23]:
ok=check_y(y,df)
if not ok:
    print(ok)

In [24]:
classification=True
if classification:
    print(df[y].levels())

[['<=50K', '>50K']]


In [25]:
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    

In [26]:
allV=get_variables_types(df)
allV

{'age': 'int',
 'workclass': 'enum',
 'fnlwgt': 'int',
 'education': 'enum',
 'educational-num': 'int',
 'marital-status': 'enum',
 'occupation': 'enum',
 'relationship': 'enum',
 'race': 'enum',
 'gender': 'enum',
 'capital-gain': 'int',
 'capital-loss': 'int',
 'hours-per-week': 'int',
 'native-country': 'enum',
 'income': 'enum'}

In [27]:
meta_data['variables']=allV

In [28]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [29]:
model_start_time = time.time()

In [30]:
aml.train(x=x,y=y,training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [36]:
meta_data['model_execution_time'] = time.time() - model_start_time

In [37]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [38]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,GBM_2_AutoML_20190414_133001,0.928814,0.277126,0.169658,0.296429,0.087870
1,GBM_1_AutoML_20190414_133001,0.928558,0.277574,0.167090,0.296428,0.087870
2,GBM_3_AutoML_20190414_133001,0.928507,0.277681,0.164332,0.296782,0.088080
3,StackedEnsemble_AllModels_AutoML_20190414_133001,0.927608,0.291877,0.176160,0.300173,0.090104
4,StackedEnsemble_BestOfFamily_AutoML_20190414_1...,0.927517,0.294431,0.174552,0.300930,0.090559
5,GBM_4_AutoML_20190414_133001,0.927496,0.279698,0.171304,0.297994,0.088800
6,GBM_5_AutoML_20190414_133001,0.926527,0.281048,0.169004,0.298709,0.089227
7,GBM_grid_1_AutoML_20190414_133001_model_3,0.925747,0.283451,0.177016,0.299681,0.089809
8,GBM_grid_1_AutoML_20190414_133001_model_4,0.923510,0.293944,0.182410,0.302863,0.091726
9,GBM_grid_1_AutoML_20190414_133001_model_2,0.923144,0.289242,0.173793,0.302537,0.091529


In [39]:
# STart best model as first model

model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [40]:
mod_best._id

'GBM_2_AutoML_20190414_133001'

In [41]:
# Get stacked ensemble  
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
se=get_stacked_ensemble(model_set)

In [42]:
print(se)

StackedEnsemble_BestOfFamily_AutoML_20190414_133001


In [43]:
if se is not None:
  mod_best=h2o.get_model(se)

In [44]:

dir(mod_best)

['F0point5',
 'F1',
 'F2',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bc',
 '_bcin',
 '_check_and_save_parm',
 '_check_targets',
 '_compute_algo',
 '_end_time',
 '_estimator_type',
 '_future',
 '_get_metrics',
 '_have_mojo',
 '_have_pojo',
 '_id',
 '_is_xvalidated',
 '_job',
 '_keyify_if_h2oframe',
 '_metrics_class',
 '_model_json',
 '_parms',
 '_plot',
 '_requires_training_frame',
 '_resolve_model',
 '_run_time',
 '_start_time',
 '_train',
 '_verify_training_frame_params',
 '_xval_keys',
 'accuracy',
 'actual_params',
 'aic',
 'algo',
 'auc',
 'base_models',
 'biases',
 'blending_frame',
 'catoffsets',
 'coef',
 'coef_norm',
 'confusion_matrix',

In [45]:
mod_best._id

'StackedEnsemble_BestOfFamily_AutoML_20190414_133001'

In [46]:
mod_best._get_metrics

<function h2o.model.model_base.ModelBase._get_metrics(o, train, valid, xval)>

In [47]:
type(mod_best)

h2o.estimators.stackedensemble.H2OStackedEnsembleEstimator

In [48]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20190414_133001
No model summary for this model


ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.07481582185367347
RMSE: 0.27352481030735304
LogLoss: 0.24746093602453897
Null degrees of freedom: 48841
Residual degrees of freedom: 48838
Null deviance: 53750.68147229072
Residual deviance: 24172.974074621066
AIC: 24180.974074621066
AUC: 0.9532884730559623
pr_auc: 0.8541503384863051
Gini: 0.9065769461119246
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.27851331516105454: 


,<=50K,>50K,Error,Rate
<=50K,33670.0,3485.0,0.0938,(3485.0/37155.0)
>50K,2066.0,9621.0,0.1768,(2066.0/11687.0)
Total,35736.0,13106.0,0.1137,(5551.0/48842.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2785133,0.7761062,252.0
max f2,0.1144129,0.8470205,327.0
max f0point5,0.6885069,0.8127353,114.0
max accuracy,0.4246866,0.8943327,199.0
max precision,0.9695915,1.0,0.0
max recall,0.0368623,1.0,392.0
max specificity,0.9695915,1.0,0.0
max absolute_mcc,0.3932602,0.7035192,211.0
max min_per_class_accuracy,0.2063390,0.8728839,282.0
max mean_per_class_accuracy,0.1763796,0.8768456,296.0


Gains/Lift Table: Avg response rate: 23.93 %, avg score: 24.02 %



...

Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3239712,0.7288937,231.0
max f2,0.1023483,0.8066681,335.0
max f0point5,0.6926612,0.7637343,113.0
max accuracy,0.5021307,0.8741452,169.0
max precision,0.9705236,1.0,0.0
max recall,0.0337119,1.0,399.0
max specificity,0.9705236,1.0,0.0
max absolute_mcc,0.4174182,0.6423761,197.0
max min_per_class_accuracy,0.1852882,0.8433051,291.0
max mean_per_class_accuracy,0.1596768,0.8453489,304.0


Gains/Lift Table: Avg response rate: 23.93 %, avg score: 23.93 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100119,0.9680113,4.1791734,4.1791734,1.0,0.9692995,1.0,0.9692995,0.0418414,0.0418414,317.9173441,317.9173441
,2,0.0200033,0.9662482,4.1620457,4.1706183,0.9959016,0.9671156,0.9979529,0.9682087,0.0415847,0.0834260,316.2045681,317.0618326
,3,0.0300152,0.9641560,4.1620807,4.1677705,0.9959100,0.9652871,0.9972715,0.9672341,0.0416702,0.1250963,316.2080707,316.7770512
,4,0.0400066,0.9608097,4.1706096,4.1684795,0.9979508,0.9626717,0.9974411,0.9660947,0.0416702,0.1667665,317.0609561,316.8479547
,5,0.0500184,0.9540303,4.1278952,4.1603560,0.9877301,0.9580829,0.9954973,0.9644910,0.0413280,0.2080945,312.7895239,316.0356041
,6,0.1000164,0.8587504,3.6298226,3.8951436,0.8685504,0.9092253,0.9320368,0.9368638,0.1814837,0.3895782,262.9822632,289.5143639
,7,0.1500143,0.7058164,2.9213141,3.5705781,0.6990172,0.7889008,0.8543742,0.8875496,0.1460597,0.5356379,192.1314113,257.0578100
,8,0.2000123,0.4904650,2.3770974,3.2722385,0.5687961,0.6015881,0.7829870,0.8160665,0.1188500,0.6544879,137.7097424,227.2238473
,9,0.3000082,0.2194531,1.6121136,2.7189013,0.3857494,0.3341085,0.6505835,0.6554248,0.1612048,0.8156926,61.2113588,171.8901277
,10,0.4000041,0.1020480,0.9925965,2.2873472,0.2375102,0.1507373,0.5473205,0.5292594,0.0992556,0.9149482,-0.7403524,128.7347167



<bound method ModelBase.coef_norm of >


In [49]:
bm=stackedensemble_df(aml_leaderboard_df)

In [50]:
bm

['GBM_2_AutoML_20190414_133001',
 'GLM_grid_1_AutoML_20190414_133001_model_1',
 'DRF_1_AutoML_20190414_133001',
 'XRT_1_AutoML_20190414_133001',
 'DeepLearning_1_AutoML_20190414_133001']

In [51]:

aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,GBM_2_AutoML_20190414_133001,0.928814,0.277126,0.169658,0.296429,0.087870
1,GBM_1_AutoML_20190414_133001,0.928558,0.277574,0.167090,0.296428,0.087870
2,GBM_3_AutoML_20190414_133001,0.928507,0.277681,0.164332,0.296782,0.088080
3,StackedEnsemble_AllModels_AutoML_20190414_133001,0.927608,0.291877,0.176160,0.300173,0.090104
4,StackedEnsemble_BestOfFamily_AutoML_20190414_1...,0.927517,0.294431,0.174552,0.300930,0.090559
5,GBM_4_AutoML_20190414_133001,0.927496,0.279698,0.171304,0.297994,0.088800
6,GBM_5_AutoML_20190414_133001,0.926527,0.281048,0.169004,0.298709,0.089227
7,GBM_grid_1_AutoML_20190414_133001_model_3,0.925747,0.283451,0.177016,0.299681,0.089809
8,GBM_grid_1_AutoML_20190414_133001_model_4,0.923510,0.293944,0.182410,0.302863,0.091726
9,GBM_grid_1_AutoML_20190414_133001_model_2,0.923144,0.289242,0.173793,0.302537,0.091529


In [52]:
#  Get best_models and coef_norm()
best_models={}
best_models=stackedensemble(mod_best)
bm=[]
if best_models is not None: 
  if 'Intercept' in best_models.keys():
    del best_models['Intercept']
  bm=list(best_models.keys())
else:
  best_models={}
  bm=stackedensemble_df(aml_leaderboard_df)   
  for b in bm:   
    best_models[b]=None

if mod_best.model_id not in bm:
    bm.append(mod_best.model_id)

In [53]:

bm

['GBM_2_AutoML_20190414_133001',
 'DRF_1_AutoML_20190414_133001',
 'XRT_1_AutoML_20190414_133001',
 'DeepLearning_1_AutoML_20190414_133001',
 'GLM_grid_1_AutoML_20190414_133001_model_1',
 'StackedEnsemble_BestOfFamily_AutoML_20190414_133001']

In [54]:
# Best of Family leaderboard

aml_leaderboard_df=aml_leaderboard_df.loc[aml_leaderboard_df['model_id'].isin(bm)]

In [55]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,GBM_2_AutoML_20190414_133001,0.928814,0.277126,0.169658,0.296429,0.087870
4,StackedEnsemble_BestOfFamily_AutoML_20190414_1...,0.927517,0.294431,0.174552,0.300930,0.090559
12,DRF_1_AutoML_20190414_133001,0.916816,0.304600,0.190084,0.306970,0.094231
13,XRT_1_AutoML_20190414_133001,0.916676,0.300848,0.177338,0.308148,0.094955
16,DeepLearning_1_AutoML_20190414_133001,0.912297,0.309177,0.189363,0.314106,0.098663
19,GLM_grid_1_AutoML_20190414_133001_model_1,0.906792,0.317975,0.197513,0.318892,0.101692


In [56]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

In [78]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_2_AutoML_20190414_133001',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_2_AutoML_20190414_133001'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_adult.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_adult.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 5},
 'fold_assignment': {'default': 'AUTO

In [57]:
top=aml_leaderboard_df.iloc[0]['model_id']
print(top)

GBM_2_AutoML_20190414_133001


In [58]:
mod_best=h2o.get_model(top)
print(mod_best._id)
print(mod_best.algo)

GBM_2_AutoML_20190414_133001
gbm


In [59]:
meta_data['mod_best']=mod_best._id
meta_data['mod_best_algo']=mod_best.algo

In [60]:
meta_data['models']=bm

In [61]:
models_path=os.path.join(run_dir,'models')
for mod in bm:
  try:   
    m=h2o.get_model(mod) 
    h2o.save_model(m, path = models_path)
  except:    
    pass

In [62]:
print(models_path)

C:\Users\pkash\L\models


In [63]:
# GBM
 
mod,mod_id=get_model_by_algo("GBM",best_models)
if mod is not None:
    try:     
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_gbm_scoring_history.csv') 
    except:
        pass   
    try:     
        stats_gbm={}
        stats_gbm=gbm_stats(mod)
        n=run_id+'_gbm_stats.json'
        dict_to_json(stats_gbm,n)
        print(stats_gbm)
    except:
        pass

{'algo': 'gbm', 'model_id': 'GBM_2_AutoML_20190414_133001', 'varimp': [('relationship', 7195.552734375, 1.0, 0.30478218389518424), ('capital-gain', 4133.49755859375, 0.5744517080455783, 0.17508264612045016), ('education', 2783.5693359375, 0.3868457974937319, 0.1179037069908138), ('occupation', 2041.1046142578125, 0.2836619631049235, 0.08645511260311377), ('marital-status', 1975.4517822265625, 0.27453787848559885, 0.08367425416679156), ('capital-loss', 1278.7720947265625, 0.17771700686974892, 0.054164977469077544), ('age', 1143.3223876953125, 0.15889292037752267, 0.04842773127814499), ('educational-num', 1043.3665771484375, 0.14500158857345424, 0.04419390083368838), ('hours-per-week', 734.0805053710938, 0.10201864018926621, 0.03109346395490157), ('native-country', 539.6834106445312, 0.07500235639526694, 0.022859381979434396), ('workclass', 374.8573303222656, 0.05209569635025758, 0.015877840105171886), ('fnlwgt', 200.03607177734375, 0.027799959108314246, 0.008472932249228834), ('gender',

In [64]:
if mod is not None:
    try:    
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_dl_scoring_history.csv') 
    except:
        pass 
    try:
        stats_dl={}
        stats_dl=dl_stats(mod)
        n=run_id+'_dl_stats.json'
        dict_to_json(stats_dl,n)
        print(stats_dl)
    except:
        pass    
    try:
        cf=mod.confusion_matrix()    
        cf_df.to_csv(run_id+'_dl_confusion_matrix.csv')
    except:
        pass

{'algo': 'gbm', 'model_id': 'GBM_2_AutoML_20190414_133001', 'varimp': [('relationship', 7195.552734375, 1.0, 0.30478218389518424), ('capital-gain', 4133.49755859375, 0.5744517080455783, 0.17508264612045016), ('education', 2783.5693359375, 0.3868457974937319, 0.1179037069908138), ('occupation', 2041.1046142578125, 0.2836619631049235, 0.08645511260311377), ('marital-status', 1975.4517822265625, 0.27453787848559885, 0.08367425416679156), ('capital-loss', 1278.7720947265625, 0.17771700686974892, 0.054164977469077544), ('age', 1143.3223876953125, 0.15889292037752267, 0.04842773127814499), ('educational-num', 1043.3665771484375, 0.14500158857345424, 0.04419390083368838), ('hours-per-week', 734.0805053710938, 0.10201864018926621, 0.03109346395490157), ('native-country', 539.6834106445312, 0.07500235639526694, 0.022859381979434396), ('workclass', 374.8573303222656, 0.05209569635025758, 0.015877840105171886), ('fnlwgt', 200.03607177734375, 0.027799959108314246, 0.008472932249228834), ('gender',

In [65]:
# DRF

mod,mod_id=get_model_by_algo("DRF",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_drf_scoring_history.csv') 
    except:
         pass  
    try: 
         stats_drf={}
         stats_drf=drf_stats(mod)
         n=run_id+'_drf_stats.json'
         dict_to_json(stats_drf,n)
         print(stats_drf)
    except:
         pass

{'algo': 'drf', 'model_id': 'DRF_1_AutoML_20190414_133001', 'varimp': [('relationship', 35362.25, 1.0, 0.15974706764874935), ('capital-gain', 32407.990234375, 0.9164572456327016, 0.14640135761527368), ('marital-status', 30051.662109375, 0.8498232468062694, 0.13575677169704095), ('occupation', 21343.14453125, 0.6035573112924093, 0.09641651063692579), ('age', 18756.22265625, 0.5304024109396319, 0.08473022982143315), ('educational-num', 16464.625, 0.465598908440498, 0.07437806032382809), ('education', 16199.431640625, 0.45809957343282737, 0.0731800635470371), ('hours-per-week', 11891.11328125, 0.3362657433067749, 0.053717466443984355), ('fnlwgt', 11234.8935546875, 0.31770867393017976, 0.05075302902691888), ('capital-loss', 8605.142578125, 0.24334262039675078, 0.03887327004234368), ('workclass', 7414.63818359375, 0.2096766518983874, 0.03349523029517496), ('native-country', 6073.3974609375, 0.17174804942947636, 0.027436247270751307), ('gender', 3037.891357421875, 0.08590775070652673, 0.0137

In [66]:
# XRT

mod,mod_id=get_model_by_algo("XRT",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_xrt_scoring_history.csv')
    except:
         pass     
    try:        
         stats_xrt={}
         stats_xrt=xrt_stats(mod)
         n=run_id+'_xrt_stats.json'
         dict_to_json(stats_xrt,n)
         print(stats_xrt)
    except:
         pass

{'algo': 'drf', 'model_id': 'XRT_1_AutoML_20190414_133001', 'varimp': [('marital-status', 43290.4375, 1.0, 0.17189797047388505), ('relationship', 39768.234375, 0.9186378487165902, 0.1579119817948777), ('capital-gain', 33771.25390625, 0.7801088613680561, 0.13409913001786222), ('occupation', 28768.361328125, 0.664543095184127, 0.11423360935458525), ('educational-num', 24060.662109375, 0.5557962335071112, 0.09554024453690191), ('age', 19889.06640625, 0.4594332502703397, 0.07897564328969191), ('education', 17284.724609375, 0.39927350259222955, 0.06863430475960373), ('hours-per-week', 10322.55078125, 0.2384487516729301, 0.040988856474608086), ('capital-loss', 9118.7587890625, 0.2106414098740051, 0.036208830855099246), ('native-country', 6771.27880859375, 0.15641511612336442, 0.0268874410130434), ('workclass', 6515.40087890625, 0.150504389772135, 0.025871399149240554), ('fnlwgt', 6235.92138671875, 0.14404847229180232, 0.024761640036824486), ('gender', 4133.4609375, 0.09548207817257565, 0.016

In [67]:
# GLM

mod,mod_id=get_model_by_algo("GLM",best_models)
if mod is not None:
    try:     
         stats_glm={}
         stats_glm=glm_stats(mod)
         n=run_id+'_glm_stats.json'
         dict_to_json(stats_glm,n)
         print(stats_glm)
    except:
         pass

{'algo': 'glm', 'model_id': 'GLM_grid_1_AutoML_20190414_133001_model_1', 'coef': {'Intercept': -8.130286272637353, 'native-country.?': 0.065729266621754, 'native-country.Cambodia': 0.5572126666010029, 'native-country.Canada': 0.5901026055722867, 'native-country.China': -0.38607416793358634, 'native-country.Columbia': -1.0542652388600449, 'native-country.Cuba': 0.23552384656444758, 'native-country.Dominican-Republic': -0.525465533873249, 'native-country.Ecuador': -0.13221664295236543, 'native-country.El-Salvador': -0.29393625283317865, 'native-country.England': 0.49511135261503186, 'native-country.France': 0.5537852277774868, 'native-country.Germany': 0.21203688770410686, 'native-country.Greece': -0.004900891142730397, 'native-country.Guatemala': -0.18203537829312894, 'native-country.Haiti': 0.21250593926755865, 'native-country.Holand-Netherlands': -0.005788764323843744, 'native-country.Honduras': 0.021601287801425277, 'native-country.Hong': -0.1714622937455509, 'native-country.Hungary'

In [68]:
# split into training and test for showing how to predict
train, test = df.split_frame([0.8])

In [69]:
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions
predictions_df=predictions_test(mod_best,test,run_id)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [70]:

predictions_df.describe()

Rows:9830
Cols:3




,predict,<=50K,>50K
type,enum,real,real
mins,,0.00414799135771271,0.000977308752271254
mean,,0.7585326521719343,0.2414673478280656
maxs,,0.9990226912477288,0.9958520086422873
sigma,,0.3098477940377674,0.3098477940377674
zeros,,0,0
missing,0,0,0
0,<=50K,0.9957339254131298,0.0042660745868701765
1,<=50K,0.9944317278314003,0.00556827216859971
2,<=50K,0.9942202915101764,0.005779708489823523


In [71]:
# Update and save meta data

meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']
  
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)

In [72]:
meta_data

{'start_time': 1555262994.0324767,
 'target': None,
 'server_path': 'C:\\Users\\pkash',
 'data_path': 'C:/Users/pkash/Downloads/adult.csv',
 'test_path': None,
 'max_models': 9,
 'run_time': 1000,
 'run_id': 'L',
 'scale': False,
 'classification': False,
 'model_path': None,
 'balance': False,
 'balance_threshold': 0.2,
 'project': None,
 'end_time': 1555263991.8573601,
 'execution_time': 997.8248834609985,
 'run_path': 'C:\\Users\\pkash\\L',
 'nthreads': 1,
 'min_mem_size': 6,
 'analysis': 0,
 'x': ['age',
  'workclass',
  'fnlwgt',
  'education',
  'educational-num',
  'marital-status',
  'occupation',
  'relationship',
  'race',
  'gender',
  'capital-gain',
  'capital-loss',
  'hours-per-week',
  'native-country'],
 'variables': {'age': 'int',
  'workclass': 'enum',
  'fnlwgt': 'int',
  'education': 'enum',
  'educational-num': 'int',
  'marital-status': 'enum',
  'occupation': 'enum',
  'relationship': 'enum',
  'race': 'enum',
  'gender': 'enum',
  'capital-gain': 'int',
  'capi

In [73]:
# Save logs
h2o.download_all_logs(dirname=logs_path, filename=logfile)

Writing H2O logs to C:\Users\pkash\L\logs\L_autoh2o_log.zip


'C:\\Users\\pkash\\L\\logs\\L_autoh2o_log.zip'

In [74]:
# Clean up
os.chdir(server_path)

In [75]:
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    print(stats_test)   
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
      print(cf)   
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      print(cf_df)    
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

In [76]:
h2o.cluster().shutdown()

H2O session _sid_963f closed.
